
# 3D field ROM example for images generation

This example shows how PyTwin can be used to load and evaluate a twin model to
visualize ROM results in the form of images with predefined views. The script
takes the inputs to evaluate the TBROM (twin builder ROM) and displays
the corresponding image.
A first image is generated using the point cloud-based ROM Viewer embedded in
the twin runtime. The second image is generated by loading the ROM results and
postprocessing the CFD mesh using PyFluent. Finally, ROM results are postprocessed
in the 3D viewer enabled by PyFluent. For more information, see the
[PyFluent-Visualization documentation](https://visualization.fluent.docs.pyansys.com).

<div class="alert alert-info"><h4>Note</h4><p>To generate snapshot files at initialization time, the ROM included in the twin
   must have its parameter ``field_data_storage_period`` set to ``0`` and its
   parameter ``store_snapshots`` set to ``1``.

   To generate images files at initialization time, the ROM included in the twin must
   have the **Embed Geometry** and **Generate Image** options enabled at export time.
   Additionally, its parameter ``viewX_storage_period`` must be set to ``0``.

   These parameters can be defined in the Twin Builder subsheet before twin compilation
   or be exposed as twin parameters.</p></div>


<img src="file://_static/TBROM_images_generation.png" width="400pt" align="center">



In [ ]:
# sphinx_gallery_thumbnail_path = '_static/TBROM_images_generation.png'

## Perform required imports
Perform required imports, which include downloading and importing the input
files.



In [ ]:
import os

import ansys.fluent.core as pyfluent
from ansys.fluent.visualization import set_config
from ansys.fluent.visualization.pyvista import Graphics
import matplotlib.image as img
import matplotlib.pyplot as plt
from pytwin import TwinModel, download_file, snapshot_to_array

twin_file = download_file("ThermalTBROM_23R1_other.twin", "twin_files", force_download=True)
cfd_file = download_file("T_Junction.cas.h5", "other_files", force_download=True)

set_config(blocking=True, set_view_on_display="isometric")

## Define ROM inputs
Define the ROM inputs.



In [ ]:
rom_inputs = {"main_inlet_temperature": 353.15, "side_inlet_temperature": 293.15}
rom_parameters = {
    "ThermalROM23R1_1_colorbar_min": 290,
    "ThermalROM23R1_1_colorbar_max": 360,
    "ThermalROM23R1_1_store_snapshots": 1,
}

## Define auxiliary functions
Define the auxiliary function for converting a ROM snapshot for data mapping
on a CFD mesh.



In [ ]:
def snapshot_to_cfd(snapshot_file, geometry_file, field_name, outputFilePath):
    """Create a Fluent Interpolation file that can be loaded in Ansys Fluent
    and map to the CFD mesh. This code shows the implementation for a single
    field of scalar data (temperature field).
    """
    res_arr = snapshot_to_array(snapshot_file, geometry_file)

    with open(outputFilePath, "w") as ipfile:
        ipfile.write("3\n")  # IP file format
        ipfile.write("3\n")  # 2D or 3D - 3D for now
        ipfile.write(str(len(res_arr)) + "\n")  # number of data
        ipfile.write("1\n")  # number of field data
        ipfile.write(field_name + "\n")  # name of field data
        for j in range(0, len(res_arr[0])):
            ipfile.write("(")
            for i in range(0, len(res_arr)):
                ipfile.write(str(res_arr[i][j]) + "\n")
            ipfile.write(")\n")

    return outputFilePath

## Load the twin runtime and generate temperature results
Load the twin runtime and generate temperature results from the TBROM.



In [ ]:
print("Loading model: {}".format(twin_file))
twin_model = TwinModel(twin_file)

twin_model.initialize_evaluation(inputs=rom_inputs, parameters=rom_parameters)

rom_name = twin_model.tbrom_names[0]
rom_directory = twin_model.get_rom_directory(rom_name)
snapshot = twin_model.get_snapshot_filepath(rom_name)
geometry = twin_model.get_geometry_filepath(rom_name)

temperature_file = snapshot_to_cfd(snapshot, geometry, "temperature", os.path.join(rom_directory, "cfd_file.ip"))

## Postprocess with image generated by point cloud-based ROM Viewer
Postprocess with the image generated by the point cloud-based ROM Viewer.



In [ ]:
view_name = twin_model.get_available_view_names(rom_name)[0]
image_filepath = twin_model.get_image_filepath(rom_name, view_name)
plt.imshow(img.imread(image_filepath))
plt.show()

## Postprocess with image generated by PyFluent
Postprocess with the image generated by PyFluent.



In [ ]:
solver = pyfluent.launch_fluent(precision="double", processor_count=2, mode="solver")
solver.file.read(file_type="case", file_name=cfd_file)
tui = solver.tui
tui.file.interpolate.read_data(temperature_file)
tui.display.objects.display("contour-1")
tui.display.save_picture(os.path.join(rom_directory, f"{view_name}_Fluent_0.000000.png"))

image = img.imread(os.path.join(rom_directory, f"{view_name}_Fluent_0.000000.png"))
plt.imshow(image)
plt.show()

## Postprocess ROM results in 3D Viewer
Postprocess the ROM results in the 3D Viewer enabled by PyFluent.



In [ ]:
graphics = Graphics(session=solver)
temperature_contour = graphics.Contours["contour-temperature"]
temperature_contour.field = "temperature"
temperature_contour.show_edges = True
temperature_contour.surfaces = [
    "convection",
    "interface-side1-wall-interface_fluid_side",
    "interface-side2-wall-interface_solid_side",
    "interface-wall1-1-1",
    "interface-wall1-1-1-shadow",
    "interface_solid_side",
    "symmetry_solid",
    "wall-19",
    "wall-20",
    "wall-solid-part_2",
    "wall_adiabatic",
    "wall_cold_end",
    "wall_hot_end",
]
temperature_contour.display()

<img src="file://_static/TBROM_fluent_viewer.png" width="400pt" align="center">



In [ ]:
solver.exit()